In [1]:
#!pip install torch==1.8.1 torchvision==0.4.1

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
import numpy as np
import os, datetime
import matplotlib.pyplot as plt

import torch as torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
torch.cuda.is_available()
np.random.seed(0) #for debugging
torch.manual_seed(0) #more debugging

In [4]:
def getGaussian(Nfft, Pk):
    xf = np.random.normal(0,1,Nfft)+1j*np.random.normal(0,1,Nfft)
    xf *= Pk
    return np.fft.irfft(xf)

def getNonGaussian(t):
    # Signal with non-Gaussian shape
    freq = np.random.uniform(200,500)
    phase = np.random.uniform(0,2*np.pi)
    sigma = 0.03
    pos = np.random.uniform(3*sigma,1-3*sigma)
    ampl = np.random.uniform (0.1,0.2)
    rfi = ampl*np.cos(phase+freq*t)*np.exp(-(t-pos)**2/(2*sigma**2))
    #print('freq: ', freq)
    #print('phase: ', phase)
    #print('pos: ', pos)
    #print('ampl: ',ampl)
    return rfi

def Gaussianize(signal):
    fsig = np.fft.rfft(signal)
    rot = np.exp(1j*np.random.uniform(0,2*np.pi,len(fsig)))
    return np.fft.irfft((fsig*rot))  

In [5]:
# define the network classes
# the decoder network
class Decoder(nn.Module):
  def __init__(self, z_dim, hidden_dim, out_dim):
    super(Decoder, self).__init__()
    self.main = nn.Sequential(
      nn.Linear(z_dim, hidden_dim),
      nn.LeakyReLU(0.02, inplace=False),

      nn.Linear(hidden_dim, hidden_dim * 2),
      nn.LeakyReLU(0.02, inplace=False),

      nn.Linear(hidden_dim * 2, out_dim, bias=False),
    )
    
  def forward(self, x):
    out = self.main(x)
    return out
    #return x-self.main(x)



# the encoder network
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, z_dim):
    super(Encoder, self).__init__()

    self.main = nn.Sequential(
      nn.Linear(input_dim, hidden_dim * 2),
      nn.LeakyReLU(0.02, inplace=False),
      nn.Linear(hidden_dim * 2, hidden_dim),
      nn.LeakyReLU(0.02, inplace=False),
      nn.Dropout(0.2),
      nn.Linear(hidden_dim, z_dim)
    )
    
  def forward(self, x):
    out = self.main(x)
    return out
    #return x-self.main(x)

In [6]:
# training constants
BATCH_SIZE = 32
Z_DIM = 16
HIDDEN_DIM = 256
LR = 0.0002
EPOCHS = 3
NCORES = 4
NTRAINING = 100000
NTEST = 10

# data parameters
N = 1024
Nfft = N//2+1
k = np.linspace(0,1,Nfft)
t = np.linspace(0,1,N)
Pk = (1+np.exp(-(k-0.5)**2/(2*0.1**2)))*np.exp(-k/0.5) #spectrum of noise

g = torch.stack([torch.from_numpy(getGaussian(Nfft, Pk)) for i in range(NTRAINING+NTEST)])
ng = torch.stack([torch.from_numpy(getNonGaussian(t)) for i in range(NTRAINING+NTEST)])

# train data
g_train_array = g[:NTRAINING]
ng_train_array = ng[:NTRAINING]

# test data
g_test_array = g[NTRAINING:]
ng_test_array = ng[NTRAINING:]

# create dataloaders
g_trainloader = DataLoader(
      torch.utils.data.TensorDataset(g_train_array),
      batch_size=BATCH_SIZE, shuffle=True, num_workers=NCORES, pin_memory=True, drop_last=True
)
ng_trainloader = DataLoader(
      torch.utils.data.TensorDataset(ng_train_array),
      batch_size=BATCH_SIZE, shuffle=True, num_workers=NCORES, pin_memory=True, drop_last=True
)

# initialize the networks
netD = Decoder(z_dim=Z_DIM, hidden_dim=HIDDEN_DIM, out_dim=N).cuda()
netE = Encoder(input_dim=N, hidden_dim=HIDDEN_DIM, z_dim=Z_DIM).cuda()

# define the optimizers
optimizer = optim.Adam([{'params': netE.parameters()}, 
                        {'params': netD.parameters()}], lr=LR, betas=(0.5, 0.999))

# loss criterion
recons_criterion = nn.MSELoss()

# training loop
iters = 0

for epoch in range(EPOCHS):
  # iterate through the dataloaders
  for i, (g, ng) in enumerate(zip(g_trainloader, ng_trainloader)):
      # set to train mode
      netE.train()
      netD.train()
      
      ng = ng[0].float().cuda()
      g = g[0].float().cuda()
      #bs = ng.shape[0]

      # new inputs
      sigs = g + ng
      gaussianized = torch.stack([torch.from_numpy(Gaussianize(sig.cpu().numpy())) for sig in sigs]).cuda().float()
      modsig = sigs + gaussianized
      
      # for the noisy input get latent representation from the encoder
      z_out = netE(modsig)

      # decode the latent representation
      recons_out = netD(z_out)

      # compute the loss
      #loss = recons_criterion(recons_out, ng)
    
      loss = recons_criterion(modsig-recons_out, gaussianized)

      # backpropagate and update the weights
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # print the training losses
      if iters % 100 == 0:
        print('[%3d/%d][%3d/%d]\tLoss: %.10f' 
            % (epoch, EPOCHS, i, len(g_trainloader), loss.item()))
        #print(sigs.grad_fn)
        #print(sigs.grad)
      iters += 1    
  # visualize generated samples
  if epoch+1==EPOCHS: ## end of final epoch
    # set to eval mode
    netD.eval()
    netE.eval()

    with torch.no_grad():
      # new inputs
      sigs = g_test_array.float().cuda() + ng_test_array.float().cuda()
      gaussianized = torch.stack([torch.from_numpy(Gaussianize(sig.cpu().numpy())) for sig in sigs]).cuda()
      modsig = (sigs + gaussianized).float()
      z_out = netE(modsig)
      recons_out = netD(z_out)

    # display a random sample from the test set
    #rand_int = np.random.randint(10)

    for test_int in range(NTEST):
        # plot/save generated samples
        plt.figure(figsize=(17,10))
        ax = plt.subplot(2,3,1)
        plt.plot(t, ng_test_array[test_int].cpu().numpy()) # ng
        ax.set_title("ng")

        ax = plt.subplot(2,3,2)
        plt.plot(t, g_test_array[test_int].cpu().numpy()) # g
        ax.set_title("g")

        ax = plt.subplot(2,3,3)
        plt.plot(t, (g_test_array[test_int].cpu() + ng_test_array[test_int].cpu()).numpy()) # g + ng 
        ax.set_title("sig = ng + g")

        ax = plt.subplot(2,3,4)
        plt.plot(t, gaussianized[test_int].cpu().numpy()) # gaussianize(ng + g)
        ax.set_title("Gauss(sig)")

        ax = plt.subplot(2,3,5)
        plt.plot(t, modsig[test_int].cpu().numpy()) # input
        ax.set_title("network input=Gauss(sig) + sig")

        ax = plt.subplot(2,3,6)
        plt.plot(t, recons_out[test_int].cpu().detach().numpy()) # output
        ax.set_title("network output (~ng)")

        save_folder = 'ae-g/'
        os.makedirs(save_folder, exist_ok=True)
        save_time = str(datetime.datetime.now()).split('.')[0].replace(' ','_').replace(':','-')
        #save_filename = str(epoch).zfill(7) + '.png'
        save_filename = save_time + '_epoch_' + str(epoch).zfill(7) + '_test_' + str(test_int) + '.png'
        save_path = os.path.join(save_folder, save_filename)
        #save_path = os.path.join(save_folder, str(epoch).zfill(7) + '.png')
        print('Saving file...{}'.format(save_path))
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
        
        #Overplot
        plt.plot(sigs[test_int].cpu().numpy())
        plt.plot(recons_out[test_int].cpu().detach().numpy())
        plt.plot(ng_test_array[test_int].cpu().numpy())
        plt.legend(['Test In','Recovered','RFI'])
    
        save_filename = save_time + '_overplot_test_' + str(test_int) + '.png'
        save_path = os.path.join(save_folder, save_filename)
        print('Saving file...{}'.format(save_path))
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
        
    #rand int seed check
    print('Numpy rand int check: ',np.random.uniform(0,1,1))
    print('Torch rand int check: ',torch.rand(1))
          


[  0/3][  0/3125]	Loss: 0.0020396314
[  0/3][100/3125]	Loss: 0.0012840308
[  0/3][200/3125]	Loss: 0.0014177836
[  0/3][300/3125]	Loss: 0.0012635158
[  0/3][400/3125]	Loss: 0.0013053633
[  0/3][500/3125]	Loss: 0.0012558863
[  0/3][600/3125]	Loss: 0.0012637856
[  0/3][700/3125]	Loss: 0.0012630094
[  0/3][800/3125]	Loss: 0.0012273312
[  0/3][900/3125]	Loss: 0.0012013507
[  0/3][1000/3125]	Loss: 0.0012612687
[  0/3][1100/3125]	Loss: 0.0012096667
[  0/3][1200/3125]	Loss: 0.0012510667
[  0/3][1300/3125]	Loss: 0.0012700292
[  0/3][1400/3125]	Loss: 0.0012602669
[  0/3][1500/3125]	Loss: 0.0012575523
[  0/3][1600/3125]	Loss: 0.0012189510
[  0/3][1700/3125]	Loss: 0.0012320076
[  0/3][1800/3125]	Loss: 0.0012188797
[  0/3][1900/3125]	Loss: 0.0012423766
[  0/3][2000/3125]	Loss: 0.0012400369
[  0/3][2100/3125]	Loss: 0.0012350675
[  0/3][2200/3125]	Loss: 0.0012383685
[  0/3][2300/3125]	Loss: 0.0011585031
[  0/3][2400/3125]	Loss: 0.0012328201
[  0/3][2500/3125]	Loss: 0.0011732216
[  0/3][2600/3125]	Los

In [7]:
from google.colab import files
#!zip -r /content/file.zip /content/ae-g
!zip -r file.zip /content/ae-g
#files.download("/content/file.zip")
files.download("file.zip")

ModuleNotFoundError: No module named 'google'